In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Research/DASH

Mounted at /content/drive
/content/drive/MyDrive/Research/DASH


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.3 MB/s eta 0:00:00


In [ ]:
metadata_fields = ['num_node', 'average_degree', 'weight_range', 'test_id', 'weight_scale', 'time_created', 'num_edge', 'weight_range/scale']
REDUCTION_FACTORS = [2, 3, 4, 6, 8, 11, 16, 23, 32, 45]
samplers = ['gurobi', 'dwave'] + [f'fl {division} 10000 0' for division in REDUCTION_FACTORS]
                                          
energies = pd.read_csv("results/energy.csv")
print(energies.columns)
opt_gap = pd.DataFrame()

for colname in energies.columns: 
    if colname in samplers: 
        opt_gap[colname] = (1 - energies[colname] / energies['gurobi']) * 100
    if colname in metadata_fields: 
        opt_gap[colname] = energies[colname]

print(opt_gap.query('num_node == 60 and average_degree == 6 and weight_scale == 32 and weight_range == 1024'))

qubits = pd.read_csv("results/qubits.csv")
print(qubits.columns)
size_expansion = pd.DataFrame()

for colname in qubits.columns: 
    if colname in samplers and colname != 'gurobi': 
        size_expansion[colname] = qubits[colname] / qubits['dwave'] 
    if colname in metadata_fields: 
        size_expansion[colname] = qubits[colname]

print(size_expansion.query('num_node == 60 and average_degree == 6 and weight_scale == 32 and weight_range == 1024'))

running_times = pd.read_csv("results/running_time.csv")
print(running_times.columns)

Index(['num_node', 'average_degree', 'weight_range', 'weight_scale', 'test_id',
       'time_created', 'num_edge', 'weight_range/scale', 'gurobi', 'neal',
       'dwave', 'fl 2 10000 0', 'fl 3 10000 0', 'fl 4 10000 0', 'fl 6 10000 0',
       'fl 8 10000 0', 'fl 11 10000 0', 'fl 16 10000 0', 'fl 23 10000 0',
       'fl 32 10000 0', 'fl 45 10000 0', 'fl 64 10000 0', 'fl 91 10000 0'],
      dtype='object')
   num_node  average_degree  weight_range  weight_scale  test_id  \
0        60               6          1024            32        1   
1        60               6          1024            32        2   
2        60               6          1024            32        3   
3        60               6          1024            32        4   
4        60               6          1024            32        5   

                 time_created  num_edge  weight_range/scale  gurobi  \
0  2023-04-12 14:32:34.439905       180                32.0     0.0   
1  2023-04-12 14:32:34.440906       180   

In [ ]:
from pandas.core.arrays.timedeltas import Tick
import matplotlib.ticker as ticker

default_sampler = {
    "sampler_names": [ 'fl 11 10000 0','dwave', 'gurobi'],
    "sampler_labels": ['HamSlicing+QA', 'QA', 'IQP (Gurobi)']
}

default_marker_sequence = [
    dict(size=20, opacity = 0.75, symbol='cross'),
    dict(size=20, opacity = 0.75, symbol='x'),
    dict(size=20, opacity = 0.75, symbol='circle-open'),
    dict(size=20, opacity = 0.75, symbol='square-open'),
    dict(size=20, opacity = 0.75, symbol='star')
]

default_color_sequence=px.colors.qualitative.Dark2
fig = px.colors.qualitative.swatches()
fig.show()
default_layout = {
    "height":600, 
    "width":800, 
    "font": dict(size=32), 
    "plot_bgcolor":'white',
    "legend": dict(
      yanchor="top",
      y=0.99,
      xanchor="left",
      x=0.01,
      bgcolor="rgba(0,0,0,0)"
      ),
    "colorway":default_color_sequence,
    "margin": dict(l=50, r=50, t=50, b=50)
  }

def plot_all_solvers(
    df = opt_gap, 
    variable_name = 'num_node', 
    variable_label = 'Number of variables', 
    quantity_label = 'Optimality gap (%)', 
    chart_types = ['mean', 'mean', 'mean'], 
    sampler_names = default_sampler["sampler_names"], 
    sampler_labels = default_sampler["sampler_labels"], 
    fig_fpath = "./plots/fig.pdf",
    layout = {},
    marker_sequence = default_marker_sequence,
    verbose = False, # Print out the info
): 
    DEFAULT_FILTERS = {
        'num_node': 60, 
        'average_degree': 6, 
        'weight_range': 1024, 
        'weight_scale': 32
    }
    filter_str = ''
    for filter_key in DEFAULT_FILTERS.keys(): 
        if filter_key != variable_name: 
            if len(filter_str) > 0: 
                filter_str += ' and '
            filter_str += filter_key + ' == ' + str(DEFAULT_FILTERS[filter_key])

    print("Filter string: ", filter_str)
    df_by_variable = df.query(filter_str)[[variable_name] + sampler_names]
    print(df_by_variable)
    
    df_mean = df_by_variable.groupby(variable_name, as_index=False).aggregate('mean')
    


    fig = sp.make_subplots(
        rows=1, 
        cols=1, 
    )

    #fig.update_annotations(font_size=72)


    for (sampler_name, sampler_label, chart_type) in zip(sampler_names, sampler_labels, chart_types): 
      fig.add_trace(
          go.Scatter(x = df_mean[variable_name], y = df_mean[sampler_name], mode='lines+markers', name=sampler_label) if chart_type == 'mean'
          else go.Scatter(x = df_by_variable[variable_name], y = df_by_variable[sampler_name], mode='markers', name=sampler_label), 
          row=1, 
          col=1
      )
    if verbose:
      print(fig.data)

    fig.update_xaxes(
        title_text=variable_label, 
        mirror=True, 
        ticks='outside', 
        showline=True, 
        linewidth=2, 
        linecolor='black', 
        row=1, 
        col=1)
    fig.update_yaxes(
        title_text=quantity_label, mirror=True, 
        ticks='outside', 
        showline=True, 
        linewidth=2, 
        linecolor='black', 
        nticks = 5,
        row=1, 
        col=1)
    for i, trace in enumerate(fig.data):
      trace.update(marker=marker_sequence[i])

    fig.update_layout(**default_layout)
    fig.update_layout(**layout)
    fig.show()
    fig.write_image(fig_fpath, format = 'pdf')

def get_sampler_name(division): 
    return 'fl {} 10000 0'.format(division)

def flatten(
    df_by_reduction_factor, 
    reduction_factors = REDUCTION_FACTORS): 

    flattened = [[], []]
    for reduction_factor in reduction_factors: 
      sampler_name = get_sampler_name(reduction_factor)
      print(df_by_reduction_factor[sampler_name].tolist())
      values = df_by_reduction_factor[sampler_name].tolist()
      flattened[0] += [reduction_factor for v in values]
      flattened[1] += values
    
    return flattened

def plot_dash_by_reduction_factor(
    dfs = [opt_gap, size_expansion], 
    quantity_labels = ['Optimality gap (%)', 'Size expansion (times)'], 
    chart_types = ['mean', 'mean'], 
    reduction_factors = REDUCTION_FACTORS, 
    log_scale = False, 
    fig_fpath = "fig.pdf", 
    sampler_names = default_sampler["sampler_names"], 
    sampler_labels = default_sampler["sampler_labels"], 
    layout = {},
    color_sequence = default_color_sequence,
    marker_sequence = default_marker_sequence,
    verbose = False # Print out the chart data for debugging
  ): 
    
    sampler_names = [get_sampler_name(reduction_factor) for reduction_factor in reduction_factors]
    
    DEFAULT_FILTERS = {
        'num_node': 60, 
        'average_degree': 6, 
        'weight_range': 1024, 
        'weight_scale': 32
    }
    
    filter_str = ''
    for filter_key in DEFAULT_FILTERS.keys(): 
        if len(filter_str) > 0: 
            filter_str += ' and '
        filter_str += filter_key + ' == ' + str(DEFAULT_FILTERS[filter_key])
            
    print("Filter string: ", filter_str)
    dfs_by_reduction_factor = [df.query(filter_str)[sampler_names] for df in dfs] # columns of qubits/energy/time are now sorted by the order of sampler_name = the order of Slicing factor
    print(dfs_by_reduction_factor)
    
    dfs_mean = [df_by_reduction_factor.aggregate('mean') for df_by_reduction_factor in dfs_by_reduction_factor]
    dfs_scatter = [flatten(df_by_reduction_factor) for df_by_reduction_factor in dfs_by_reduction_factor]

    for df_mean in dfs_mean: 
      print("df_mean: ", df_mean.values.flatten().tolist())
    
    fig = sp.make_subplots(
        rows=1, 
        cols=1, 
        specs=[[{"secondary_y": True}]]
    )

    for i, (df_mean, df_scatter, chart_type) in enumerate(zip(dfs_mean, dfs_scatter, chart_types)): 
      fig.add_trace(
          go.Scatter(
              x = reduction_factors if chart_type == 'mean' else df_scatter[0], 
              y=df_mean.values.flatten().tolist() if chart_type == 'mean' else df_scatter[1], 
              mode='lines+markers' if chart_type == 'mean' else 'markers', 
              name=quantity_labels[i]
            ), 
          row=1, 
          col=1, 
          secondary_y = (i == 1)
      )

    fig.update_xaxes(
        title_text='Slicing factor', 
        mirror=True, 
        ticks='outside', 
        showline=True, 
        linewidth=2, 
        linecolor='black', 
        row=1, 
        col=1)
    for i in range(2): 
      fig.update_yaxes(
          title_text=quantity_labels[i], 
          ticks='outside', 
          showline=True, 
          linewidth=2,                   
          nticks=5,
          linecolor='black',
          row=1, 
          col=1, 
          secondary_y = (i == 1)
        )
    for i, trace in enumerate(fig.data):
      trace.update(marker=marker_sequence[i])
    # if log_scale: 
    #   fig.update_xaxes(type='log', dtick=0.30102999566)
    #   fig.update_yaxes(type='log', dtick=0.30102999566, secondary_y = False)
    #   fig.update_yaxes(type='log', dtick=0.30102999566, secondary_y = True)
    fig.update_layout(**default_layout)
    fig.update_layout(**layout)

    

    if verbose:
      print(fig.data)

    fig.show()
    fig.write_image(fig_fpath, format = 'pdf')


plot_dash_by_reduction_factor(
    fig_fpath = "plots/tradeoff_log_scale.pdf", 
    log_scale=True,
    layout = {"colorway":default_color_sequence[4:], 
              "legend": dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.09
                )},
    marker_sequence= default_marker_sequence[3:]
)

Filter string:  num_node == 60 and average_degree == 6 and weight_range == 1024 and weight_scale == 32
[   fl 2 10000 0  fl 3 10000 0  fl 4 10000 0  fl 6 10000 0  fl 8 10000 0  \
0      9.039693      6.862996      3.790013      1.177977      1.382843   
1      6.547541      4.799791      3.417243      2.217295      1.043433   
2      7.716437      8.469260      5.269762      4.140527      2.979925   
3      6.019152      3.939808      5.608755      4.186047      2.270862   
4      8.451838      5.071103      3.380735      4.078347      1.958680   

   fl 11 10000 0  fl 16 10000 0  fl 23 10000 0  fl 32 10000 0  fl 45 10000 0  
0       0.845070       1.613316       0.000000       0.742638       2.253521  
1       0.730403       0.521716       0.652146       0.000000       1.069519  
2       2.227102       1.348808       1.442911       1.537014       3.826851  
3       1.477428       1.504788       1.778386       0.656635       1.149111  
4       1.958680       1.368393       0.348806    

In [ ]:

# print(plotly.io.orca.config)

plot_all_solvers(
    df=opt_gap, 
    variable_name='num_node', 
    variable_label='Number of variables', 
    quantity_label = 'Optimality gap (%)', 
    chart_types = ['scatter', 'scatter', 'mean'], 
    fig_fpath="plots/opt_gap_num_node.pdf",
    )

Filter string:  average_degree == 6 and weight_range == 1024 and weight_scale == 32
     num_node  fl 11 10000 0      dwave  gurobi
0          60       0.845070  14.443022     0.0
1          60       0.730403  13.121169     0.0
2          60       2.227102  14.491844     0.0
3          60       1.477428  11.627907     0.0
4          60       1.958680  14.193721     0.0
..        ...            ...        ...     ...
105       180      10.325745  39.818891     0.0
106       180      13.807829  43.416370     0.0
107       180       6.286155  40.429361     0.0
108       180      12.695593  36.474206     0.0
109       180      11.884136  41.662155     0.0

[70 rows x 4 columns]


In [ ]:
plot_all_solvers(
    df=opt_gap, 
    variable_name='average_degree', 
    variable_label='Average degree', 
    quantity_label = 'Optimality gap (%)', 
    chart_types = ['scatter', 'scatter', 'mean'], 
    fig_fpath="plots/opt_gap_avg_deg.pdf",
    layout = {"legend": dict(font=dict(size=32), bgcolor='rgba(0,0,0,0)')}
)

Filter string:  num_node == 60 and weight_range == 1024 and weight_scale == 32
    average_degree  fl 11 10000 0      dwave  gurobi
0                6       0.845070  14.443022     0.0
1                6       0.730403  13.121169     0.0
2                6       2.227102  14.491844     0.0
3                6       1.477428  11.627907     0.0
4                6       1.958680  14.193721     0.0
25              12       1.476937  12.701659     0.0
26              12       2.729377  13.939320     0.0
27              12       0.710765  14.192365     0.0
28              12       2.220145  19.023136     0.0
29              12       1.704609  11.281821     0.0
30              10       0.000000  20.071006     0.0
31              10       2.573234  14.727168     0.0
32              10       1.336303  16.176298     0.0
33              10       0.431377  21.864003     0.0
34              10       1.136627  11.598237     0.0
35               8       1.858544  14.765101     0.0
36               8  

In [ ]:
plot_all_solvers(
    df=running_times, 
    variable_name='num_node', 
    variable_label='Number of variables', 
    quantity_label = 'Running time (s)', 
    chart_types=['mean', 'mean', 'mean'], 
    fig_fpath="plots/run_time_num_node.pdf"
)

Filter string:  average_degree == 6 and weight_range == 1024 and weight_scale == 32
     num_node  fl 11 10000 0  dwave    gurobi
0          60       0.108043   0.06  0.028239
1          60       0.108076   0.06  0.023222
2          60       0.117543   0.06  0.059876
3          60       0.118944   0.06  0.029593
4          60       0.120349   0.06  0.029461
..        ...            ...    ...       ...
105       180       0.266090   0.06  0.525614
106       180       0.526195   0.06  0.517685
107       180       0.153557   0.06  0.501383
108       180       0.314534   0.06  0.327372
109       180       0.534895   0.06  0.375902

[70 rows x 4 columns]


In [ ]:
plot_all_solvers(
    df=running_times, 
    variable_name='average_degree', 
    variable_label='Average degree', 
    quantity_label = 'Running time (s)', 
    chart_types=['mean', 'mean', 'mean'], 
    fig_fpath="plots/run_time_avg_deg.pdf"
)

Filter string:  num_node == 60 and weight_range == 1024 and weight_scale == 32
    average_degree  fl 11 10000 0  dwave    gurobi
0                6       0.108043   0.06  0.028239
1                6       0.108076   0.06  0.023222
2                6       0.117543   0.06  0.059876
3                6       0.118944   0.06  0.029593
4                6       0.120349   0.06  0.029461
25              12       0.133857   0.06  0.510417
26              12       0.146416   0.06  0.532535
27              12       0.129960   0.06  0.181641
28              12       0.130601   0.06  0.160549
29              12       0.129768   0.06  0.315457
30              10       0.126701   0.06  0.113574
31              10       0.126088   0.06  0.103551
32              10       0.130449   0.06  0.123215
33              10       0.125183   0.06  0.100447
34              10       0.106203   0.06  0.157145
35               8       0.126419   0.06  0.082941
36               8       0.122309   0.06  0.103371
37 

In [ ]:
plot_all_solvers(
    df=qubits, 
    variable_name='num_node', 
    variable_label='Number of variables', 
    quantity_label = 'Embedded sizes (qubits)', 
    chart_types=['mean', 'mean',], 
    sampler_names=[ 'fl 11 10000 0', 'dwave'], 
    sampler_labels=[ 'HamSlicing+QA','QA'], 
    fig_fpath="plots/qubits_num_node.pdf"
)

Filter string:  average_degree == 6 and weight_range == 1024 and weight_scale == 32
     num_node  fl 11 10000 0  dwave
0          60            311    147
1          60            344    138
2          60            281    150
3          60            293    149
4          60            346    156
..        ...            ...    ...
105       180           1289    978
106       180           1567    840
107       180           1251    890
108       180           1571    905
109       180           1897    867

[70 rows x 3 columns]


In [ ]:
plot_all_solvers(
    df=qubits, 
    variable_name='average_degree', 
    variable_label='Average degree', 
    quantity_label = 'Embeded size (qubits)', 
        chart_types=['mean', 'mean'], 
    sampler_names=['fl 11 10000 0','dwave'], 
    sampler_labels=['HamSlicing+QA', 'QA'], 
    fig_fpath="plots/qubits_avg_deg.pdf"
)

Filter string:  num_node == 60 and weight_range == 1024 and weight_scale == 32
    average_degree  fl 11 10000 0  dwave
0                6            311    147
1                6            344    138
2                6            281    150
3                6            293    149
4                6            346    156
25              12            522    289
26              12            509    270
27              12            460    267
28              12            477    292
29              12            486    284
30              10            426    240
31              10            467    245
32              10            426    235
33              10            396    220
34              10            442    239
35               8            376    186
36               8            370    195
37               8            388    206
38               8            375    198
39               8            408    204
40               4            281     95
41               4 

In [ ]:
plot_dash_by_reduction_factor(
    fig_fpath = "plots/tradeoff_log_scale.pdf", 
    layout = {"colorway":default_color_sequence[4:], 
              "legend": dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.09
                )},
    marker_sequence= default_marker_sequence[3:],
    log_scale=True
)


Filter string:  num_node == 60 and average_degree == 6 and weight_range == 1024 and weight_scale == 32
[   fl 2 10000 0  fl 3 10000 0  fl 4 10000 0  fl 6 10000 0  fl 8 10000 0  \
0      9.039693      6.862996      3.790013      1.177977      1.382843   
1      6.547541      4.799791      3.417243      2.217295      1.043433   
2      7.716437      8.469260      5.269762      4.140527      2.979925   
3      6.019152      3.939808      5.608755      4.186047      2.270862   
4      8.451838      5.071103      3.380735      4.078347      1.958680   

   fl 11 10000 0  fl 16 10000 0  fl 23 10000 0  fl 32 10000 0  fl 45 10000 0  
0       0.845070       1.613316       0.000000       0.742638       2.253521  
1       0.730403       0.521716       0.652146       0.000000       1.069519  
2       2.227102       1.348808       1.442911       1.537014       3.826851  
3       1.477428       1.504788       1.778386       0.656635       1.149111  
4       1.958680       1.368393       0.348806    

In [ ]:

plot_dash_by_reduction_factor(
    fig_fpath = "plots/tradeoff_linear_scale.pdf", 
    layout = {"colorway":default_color_sequence[4:], 
              "legend": dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.09
                )},
    marker_sequence= default_marker_sequence[3:],
    log_scale=False
)

Filter string:  num_node == 60 and average_degree == 6 and weight_range == 1024 and weight_scale == 32
[   fl 2 10000 0  fl 3 10000 0  fl 4 10000 0  fl 6 10000 0  fl 8 10000 0  \
0      9.039693      6.862996      3.790013      1.177977      1.382843   
1      6.547541      4.799791      3.417243      2.217295      1.043433   
2      7.716437      8.469260      5.269762      4.140527      2.979925   
3      6.019152      3.939808      5.608755      4.186047      2.270862   
4      8.451838      5.071103      3.380735      4.078347      1.958680   

   fl 11 10000 0  fl 16 10000 0  fl 23 10000 0  fl 32 10000 0  fl 45 10000 0  
0       0.845070       1.613316       0.000000       0.742638       2.253521  
1       0.730403       0.521716       0.652146       0.000000       1.069519  
2       2.227102       1.348808       1.442911       1.537014       3.826851  
3       1.477428       1.504788       1.778386       0.656635       1.149111  
4       1.958680       1.368393       0.348806    